In [1]:
###Import all modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import heapq
import os
import re
from bs4 import BeautifulSoup as htmltotxt

import sklearn.preprocessing as skl_pre
import sklearn.linear_model as skl_lm
import sklearn.discriminant_analysis as skl_da
import sklearn.neighbors as skl_nb

In [25]:
###Creates dataframe with features

#master_array = [] #append results to array to create data frame
#for filename in os.listdir('season/'):
#  split_name = filename.split('.') #obtain the season and episodes
#  season = split_name[0]   
#  episode = split_name[1]
#  #TODO method to get each line of the script
#  master_array.append([season, episode])
master_dict = dict()
feature_dict = dict()

top5monica = ['cooking','chandler?','work','joey','boots']
top5joey = ['dude','casting','chandler?','agent','director']
top5phoebe = ['la','massage','smelly','yay!','buffay']
top5rachel = ['mon','barry','joshua','plane','ugh!']
top5ross = ['julie','dude','mon','justjust','elizabeth']
top5chandler = ['tulsa','kathy','joe!','gym','ring!']

single_words = ["dude","crazy"]

Features = ["Number of lines","Number of words","Words per line","Exclamation marks","Question marks","Characters",
           "top5monica","top5joey","top5phoebe","top5rachel","top5ross","top5chandler",
           "monica","joey","phoebe","rachel","ross","chandler"]
Features.extend(single_words)
for feat in Features:
    master_dict[feat] = []
    feature_dict[feat] = 0

chars = ['monica','joey','phoebe','rachel','ross','chandler']

words_char = {'monica': []
              ,'joey': []
              ,'phoebe': []
              ,'rachel': []
              ,'ross': []
              ,'chandler':[]}

for filename in os.listdir('season/'):  
    f = open('season/'+filename, 'r')

    data = htmltotxt(f.read()).get_text() #reads f as html file and converts into txt
    
    name_dict = dict()
    for char in chars:
        name_dict[char] = feature_dict.copy()

    pattern = re.compile(r'\s(?=\w+(?=:))') #store the regex
    result = re.split(pattern, data) # split the script where our pattern matched (pink dot)

    episode_array=[]
    for item in result:
        split_line = item.split(':')
        try:
            character = split_line[0]
            speech = split_line[1]
            episode_array.append([character, speech])
        except:
            pass


    i = 0
    while(len(episode_array) > i):
        if episode_array[i][0].lower() not in chars:
            del episode_array[i]
        else:
            episode_array[i][0] =episode_array[i][0].lower()
            episode_array[i][1] =episode_array[i][1].lower()
            #episode_array[i][1] = re.sub('[()]', '', episode_array[i][1])
            #episode_array[i][1] = re.sub('-[^-]+-', '', episode_array[i][1]) #removes everything within --        
            episode_array[i][1] = episode_array[i][1].replace("-","").replace("\n"," ").replace("\xa0"," ").replace(",","").replace(".","").replace(":","").replace("'","").replace("opening credits","").replace("closing credits","").replace("ending credits","").replace("commercial break","").replace("\x92","")
            episode_array[i][1] = re.sub("[\(\[].*?[\)\]]", "", episode_array[i][1]) #removes everything withing () and []
            episode_array[i][1] = re.sub('\[.*', '', episode_array[i][1])
            episode_array[i][1] = re.sub('\(.*', '', episode_array[i][1])
            episode_array[i][1] = episode_array[i][1].strip()
            if len(episode_array) == i+1:
                episode_array[i][1] = episode_array[i][1][:len(episode_array[i][1])-3]
                episode_array[i][1] = episode_array[i][1].strip()

            words = episode_array[i][1].split()

            name_dict[episode_array[i][0]]["Number of lines"] +=1
            name_dict[episode_array[i][0]]["Number of words"] +=len(words)
            name_dict[episode_array[i][0]]["Exclamation marks"] += episode_array[i][1].count('!')
            name_dict[episode_array[i][0]]["Question marks"] += episode_array[i][1].count('?')
            
            episode_array[i][1].replace('chan','chandler').replace('chandlers','chandler').replace('mnca','monica').replace('phoe','phoebe').replace('pheebs','phoebe').replace('rache','rachel').replace('rach','rachel')
            name_dict[episode_array[i][0]]["monica"] += episode_array[i][1].count('monica')
            name_dict[episode_array[i][0]]["joey"] += episode_array[i][1].count('joey')
            name_dict[episode_array[i][0]]["phoebe"] += episode_array[i][1].count('phoebe')
            name_dict[episode_array[i][0]]["rachel"] += episode_array[i][1].count('rachel')
            name_dict[episode_array[i][0]]["ross"] += episode_array[i][1].count('ross')
            name_dict[episode_array[i][0]]["chandler"] += episode_array[i][1].count('chandler')
            #name_dict[episode_array[i][0]]["top5monica"] += len(list(set(words).intersection(top5monica)))
            #name_dict[episode_array[i][0]]["top5joey"] += len(list(set(words).intersection(top5joey)))
            #name_dict[episode_array[i][0]]["top5phoebe"] += len(list(set(words).intersection(top5phoebe)))
            #name_dict[episode_array[i][0]]["top5rachel"] += len(list(set(words).intersection(top5rachel)))
            #name_dict[episode_array[i][0]]["top5ross"] += len(list(set(words).intersection(top5ross)))
            #name_dict[episode_array[i][0]]["top5chandler"] += len(list(set(words).intersection(top5chandler)))
            for singword in single_words:
                name_dict[episode_array[i][0]][singword] += episode_array[i][1].count(singword)
            
            i +=1

    for j in chars:
        if name_dict[j]["Number of lines"] != 0:
            name_dict[j]["Words per line"] = name_dict[j]["Number of words"]/name_dict[j]["Number of lines"]

        master_dict["Number of lines"].append(name_dict[j]["Number of lines"])
        master_dict["Number of words"].append(name_dict[j]["Number of words"])
        master_dict["Words per line"].append(name_dict[j]["Words per line"])
        master_dict["Exclamation marks"].append(name_dict[j]["Exclamation marks"])
        master_dict["Question marks"].append(name_dict[j]["Question marks"])
        master_dict["Characters"].append(j)
        master_dict["monica"].append(name_dict[j]["monica"])
        master_dict["joey"].append(name_dict[j]["joey"])
        master_dict["phoebe"].append(name_dict[j]["phoebe"])
        master_dict["rachel"].append(name_dict[j]["rachel"])
        master_dict["ross"].append(name_dict[j]["ross"])
        master_dict["chandler"].append(name_dict[j]["chandler"])
        for singword in single_words:
            master_dict[singword].append(name_dict[j][singword])
        
        
        #master_dict["top5monica"].append(name_dict[j]["top5monica"])
        #master_dict["top5joey"].append(name_dict[j]["top5joey"])
        #master_dict["top5phoebe"].append(name_dict[j]["top5phoebe"])
        #master_dict["top5rachel"].append(name_dict[j]["top5rachel"])
        #master_dict["top5ross"].append(name_dict[j]["top5ross"])
        #master_dict["top5chandler"].append(name_dict[j]["top5chandler"])

print(master_dict)

epinfo = pd.DataFrame.from_dict(master_dict, orient='index')
epinfo.transpose().to_csv('epinfo.csv', index=False)        

    

{'Number of lines': [73, 41, 19, 49, 47, 39, 28, 9, 14, 38, 63, 16, 52, 28, 38, 26, 33, 36, 48, 24, 32, 43, 40, 32, 33, 36, 23, 37, 40, 35, 34, 30, 10, 15, 22, 58, 23, 27, 25, 43, 53, 25, 34, 11, 16, 18, 48, 37, 48, 32, 20, 35, 36, 26, 23, 19, 49, 18, 29, 31, 41, 32, 41, 19, 62, 42, 23, 32, 43, 37, 52, 38, 18, 43, 27, 27, 12, 35, 21, 21, 12, 19, 22, 40, 40, 32, 30, 33, 38, 36, 17, 27, 38, 23, 32, 52, 55, 27, 32, 50, 34, 21, 39, 26, 35, 71, 63, 32, 37, 17, 24, 58, 50, 18, 31, 23, 18, 57, 21, 42, 69, 22, 14, 31, 29, 20, 48, 22, 43, 34, 38, 57, 20, 25, 19, 22, 57, 21, 38, 41, 24, 53, 37, 35, 27, 22, 33, 46, 45, 27, 32, 25, 21, 35, 23, 15, 30, 24, 29, 28, 22, 45, 16, 13, 33, 47, 45, 17, 38, 30, 25, 31, 56, 63, 26, 34, 34, 24, 31, 35, 0, 18, 0, 1, 38, 0, 0, 18, 0, 0, 49, 0, 32, 36, 50, 26, 37, 31, 0, 34, 0, 0, 39, 0, 29, 19, 28, 36, 40, 20, 47, 63, 48, 55, 49, 69, 43, 21, 16, 29, 44, 36, 46, 23, 19, 52, 55, 19, 34, 46, 29, 28, 22, 42, 42, 38, 33, 18, 40, 37, 35, 17, 20, 31, 30, 37, 30, 59, 

In [29]:
###Create csv
epinfo.transpose().to_csv('epinfo.csv', index=False)

In [27]:
###Train model
url = 'epinfo.csv'
data = pd.read_csv(url)
#print(data)
x = []
loops = 10
xtraintest = ['Number of lines','Number of words','Words per line','Exclamation marks','Question marks',
                    'monica','joey','phoebe','rachel','ross','chandler','dude','crazy']
model = skl_lm.LogisticRegression(solver = 'lbfgs', max_iter = 4000)
for i in range(loops):

    trainI = np.random.choice(data.shape[0], size = int(len(data)*.9), replace = False)
    trainIndex = data.index.isin(trainI)
    train = data.iloc[trainIndex] #training set
    test = data.iloc[~trainIndex] #test set

    #x='Number of lines','Number of words','Different words used','Words per line','Different words ratio','Exclamation marks','Question marks','Most common 10','Most common 30','Most common 50'
    
    X_train = train[xtraintest]
    Y_train = train['Characters']
    X_test = test[xtraintest]
    Y_test = test['Characters']

    model.fit(X_train,Y_train)

    x.append(np.mean(model.predict(X_test) == Y_test))
    print(x[i])

xmean = np.mean(x)

print("Total average:",xmean)


0.41605839416058393
0.40875912408759124
0.4744525547445255
0.40145985401459855
0.3795620437956204
0.34306569343065696
0.40145985401459855
0.4306569343065693
0.4233576642335766
0.46715328467153283
Total average: 0.4145985401459854


###### 

['hej', 'hej']


In [56]:
#def Diff(li1, li2):
#    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2]
#    return li_dif

commonwords = list(set(words_char['monica']) & set(words_char['joey']) & set(words_char['phoebe']) & set(words_char['rachel']) & set(words_char['ross']) & set(words_char['chandler']))
#monicauniqeDiff(words_char['monica'], commonwords)
monicaunique = list(set(words_char['chandler']).symmetric_difference(set(commonwords)))

commonmonica = dict()
for i in words_char['chandler']:
    if i in monicaunique:
        if i in commonmonica:
            commonmonica[i] +=1
        else:        
            commonmonica[i] = 1
commonmonica = sorted(commonmonica.items(), key=lambda item: item[1])
print(commonmonica)
top5monica = ['cooking','chandler?','work','joey','boots']
top5joey = ['dude','casting','chandler?','agent','director']
top5phoebe = ['la','massage','smelly','yay!','buffay']
top5rachel = ['mon','barry','joshua','plane','ugh!']
top5ross = ['julie','dude','mon','justjust','elizabeth']
top5chandler = ['tulsa','kathy','joe!','gym','ring!']


[('hump?', 1), ('hump', 1), ('hairpiece?', 1), ('cafeteria', 1), ('tuna', 1), ('salad?', 1), ('decide!', 1), ('diary', 1), ('lshaped', 1), ('bracket', 1), ('finished!', 1), ('spoil', 1), ('mento', 1), ('closeness', 1), ('point!', 1), ('freezer!', 1), ('angelas', 1), ('screamer', 1), ('andrea', 1), ('invade', 1), ('poland', 1), ('input', 1), ('gippetto', 1), ('"once', 1), ('boy"', 1), ('upbeat', 1), ('retailers', 1), ('serving', 1), ('liza', 1), ('minelli', 1), ('standup', 1), ('floyd', 1), ('concerts', 1), ('traffic', 1), ('episode', 1), ('threes', 1), ('misunderstanding', 1), ('realised', 1), ('cruel', 1), ('parent', 1), ('flatness', 1), ('childs', 1), ('ew!', 1), ('thighmaster!', 1), ('doy', 1), ('dinah?', 1), ('lasagne', 1), ('boys?', 1), ('perspective', 1), ('mon?', 1), ('presumably', 1), ('preferred', 1), ('enterprise', 1), ('leatherwearing', 1), ('alcoholics', 1), ('die?"', 1), ('huh?"', 1), ('"smoke', 1), ('away"', 1), ('smokes', 1), ('cell', 1), ('puff', 1), ('complete', 1), ('

In [49]:
top5monica

['cooking', 'chandler?', '2', 'work!', 'joey?']

In [16]:
name_dict['chandler']

{'Number of lines': 83,
 'Number of words': 685,
 'Words per line': 8.25301204819277,
 'Exclamation marks': 29,
 'Question marks': 29,
 'Characters': 0,
 'top5monica': 1,
 'top5joey': 0,
 'top5phoebe': 0,
 'top5rachel': 0,
 'top5ross': 0,
 'top5chandler': 0}

In [20]:
Features = ["Number of lines","Number of words","Words per line","Exclamation marks","Question marks","Characters",
           "top5monica","top5joey","top5phoebe","top5rachel","top5ross","top5chandler",
           "monica","joey","phoebe","rachel","ross","chandler"]
names = ["dude","chick"]
Features.extend(names)
Features

['Number of lines',
 'Number of words',
 'Words per line',
 'Exclamation marks',
 'Question marks',
 'Characters',
 'top5monica',
 'top5joey',
 'top5phoebe',
 'top5rachel',
 'top5ross',
 'top5chandler',
 'monica',
 'joey',
 'phoebe',
 'rachel',
 'ross',
 'chandler',
 'dude',
 'chick']